# Using the database generator study in FUSE

In [ ]:
using Revise
using FUSE
using Plots;
FUSE.logging(Logging.Info; actors=Logging.Error);


In [ ]:
ini, act = FUSE.case_parameters(:ITER; init_from=:scalars)
act.ActorPedestal.density_match=:ne_line

ini.core_profiles.ne_setting = :greenwald_fraction
ini.core_profiles.ne_value = 0.2 ↔ [0.2, 1.0]

# as an example let's look at different impurity species and what it will do to the radiation
ini.core_profiles.impurity = :Kr ↔ (:Kr, :Ne, :Xe)
ini.core_profiles.plasma_mode = :H_mode ↔ (:H_mode, :L_mode) 
ini.core_profiles.zeff = 1.1 ↔ [1.1,10.]

In [ ]:
# Testing a sample case to make sure it will run nicely later on
dd = IMAS.dd()
ini_test = rand(ini)
FUSE.init(dd,ini_test,act)
FUSE.digest(dd)

In [ ]:
sty, _ = FUSE.study_parameters(:DatabaseGenerator)
sty.server = "localhost"
sty.n_workers = 4
sty.file_save_mode = :safe_write
sty.save_folder = mktempdir()#<Your safe folder>
sty.n_simulations = 10

In [ ]:
study = FUSE.StudyDatabaseGenerator(sty, ini, act)

using Distributed
@everywhere import FUSE
@everywhere ProgressMeter = FUSE.ProgressMeter

@everywhere function workflow_DatabaseGenerator(dd::FUSE.IMAS.dd, ini::FUSE.ParametersAllInits, act::FUSE.ParametersAllActors)
    FUSE.init(dd, ini, act)
    return nothing
end

study.workflow = workflow_DatabaseGenerator



In [ ]:
FUSE.run(study)

In [ ]:
# We see that the total radiation is heavily correlated with zeff
scatter(study.dataframe.zeff_ped,abs.(study.dataframe.Prad_tot),xlabel = "zeff", ylabel="Total radiation [MW]",yscale=:log10,label=nothing)

In [ ]:
# Let's load in a single case to look at it
dd,ini,act = FUSE.load(joinpath(splitpath(study.dataframe.dir[end])[1:end-1]))

FUSE.digest(dd)

In [ ]:
# If we wanted to do further analysis on this one case we can work with that like:
FUSE.ActorFluxMatcher(dd,act;do_plot=true);
# Maybe this inspired us to redo the database run but make sure run the fluxmatcher as well by editing the workflow_DatabaseGenerator function

In [ ]:
# This is another way of running the database generator , instead of randomly sampling the ranges and options we can parameterically scan by passing a list of inis
inis = [deepcopy(ini),deepcopy(ini)]
inis[1].core_profiles.ne_value = 0.8

sty.save_folder = mktempdir()

study = FUSE.StudyDatabaseGenerator(sty, ini, act)
study.workflow = workflow_DatabaseGenerator


using Distributed
@everywhere import FUSE
@everywhere ProgressMeter = FUSE.ProgressMeter

@everywhere function workflow_DatabaseGenerator(dd::FUSE.IMAS.dd, ini::FUSE.ParametersAllInits, act::FUSE.ParametersAllActors)
    FUSE.init(dd, ini, act)
    return nothing
end

FUSE.run(study)
